# Horizontal sections

In [1]:
DATADIR='data/' # Directory with the data
FIGURESDIR='figures/' # Figures produced

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import os

import shapely.geometry as geometry

from sklearn.neighbors import KNeighborsClassifier

In [2]:
ELEVATIONS=[0,-5,-10,-15,-20,-25,-30,-40,-50,-60,-80,-100]

# Plot parameters
SCATTER=True
OPACITY=0.6
FRAME=500

In [3]:
dat=pd.read_excel(DATADIR+'hsd new basements.xls')
dat=dat.drop(columns=['Codi'])

ax=[dat.UTM_X.min()-FRAME,dat.UTM_X.max()+FRAME,dat.UTM_Y.min()-FRAME,dat.UTM_Y.max()+FRAME]
minx_rounded = 1000 * round(ax[0]/1000)
maxx_rounded = 1000 * round(ax[1]/1000)

xx, yy = np.meshgrid(np.linspace(ax[0],ax[1],300),np.linspace(ax[2],ax[3],300))
C=np.zeros(xx.shape,dtype = float)

contour=pd.read_csv(DATADIR+'deltacontourn.csv')
contour=contour.drop(columns=['Cota'])
polygon=geometry.Polygon(zip(contour['UTMX'],contour['UTMY']))

outside=[] # Points outside the LRD contour
for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        if polygon.distance(geometry.Point(xx[i,j],yy[i,j]))>1:
            outside.append((i,j))
            
xpol,ypol = polygon.exterior.xy

# TODO ¿mas simple?
cm = mpl.colors.ListedColormap(['cyan','black','yellow','black','gray','black','saddlebrown'])
cm = cm(np.arange(cm.N))
cm[:,-1]=np.array([OPACITY,0,OPACITY,0,OPACITY,0,OPACITY])
cm = mpl.colors.ListedColormap(cm)
epsilon=10**(-15)
levels = [-0.5,0.5,1-epsilon,1+epsilon,1.5,2+epsilon,2.5,3.5]
norm=mpl.colors.BoundaryNorm(levels, 7)

for E in ELEVATIONS:
    print('Eleveation:',E)

    df=dat[dat.Cota==E] # We take the chosen elevation

    gravels=df[df.Clase=='gravillas y gravas']
    sands=df[df.Clase=='arenas']
    clays=df[df.Clase=='arcillas y limos']
    basements=df[df.Valor=='S']
    
    X=np.concatenate((np.array(gravels['UTM_X']),np.array(sands['UTM_X']),
                      np.array(clays['UTM_X']),np.array(basements['UTM_X'])))
    Y=np.concatenate((np.array(gravels['UTM_Y']),np.array(sands['UTM_Y']),
                      np.array(clays['UTM_Y']),np.array(basements['UTM_Y'])))
    Z=np.concatenate((np.array(gravels['Cota']),np.array(sands['Cota']),
                      np.array(clays['Cota']),np.array(basements['Cota'])))
    
    clas=[0]*len(gravels)+[1]*len(sands)+[2]*len(clays)+[3]*len(basements)
        
    # TODO: por aquí    
        
    for DEPTH_PENALTY in DEPTH_PENALTIES:
        print('DEPTH_PENALTY:',DEPTH_PENALTY)
        
        data=list(zip(X,Y,Z*DEPTH_PENALTY))
        
        for N in Ns:
            print('N:',N)
            classifier=KNeighborsClassifier(N,weights='distance')
            classifier.fit(data,clas)

            for i in range(xx.shape[0]):
                d=list(zip(xx[i],yy[i],[COTA]*len(xx[i])))
                C[i]=classifier.predict(d)

            for (i,j) in outside:
                C[i,j]=np.nan

            plt.contourf(xx,yy,C, levels, cmap=cm, norm=norm, antialiased = True)
            
            gravillas0=gravillas[gravillas.Cota==COTA]
            arenas0=arenas[arenas.Cota==COTA]
            arcillas0=arcillas[arcillas.Cota==COTA]
            suelos0=suelos[suelos.Cota==COTA]

            if SCATTER:
                plt.scatter(gravillas0['UTM_X'], gravillas0['UTM_Y'], c='blue', s=5, alpha=0.8, label='gravillas')
                plt.scatter(arenas0['UTM_X'], arenas0['UTM_Y'], c='orange', s=5, alpha=0.7, label='arenas')
                plt.scatter(arcillas0['UTM_X'], arcillas0['UTM_Y'], c='black', s=5, alpha=0.6, label='arcillas')
                plt.scatter(suelos0['UTM_X'], suelos0['UTM_Y'], c='brown', s=5, alpha=0.6, label='suelos')

            plt.plot(xpol, ypol, alpha=0.6, color='red', linewidth=1.5, linestyle='dashed')    
            
            plt.xlabel('UTM_X')
            plt.ylabel('UTM_Y')

            plt.xticks(np.arange(minx_rounded, maxx_rounded, step=3000))

            plt.title("Height "+str(COTA)+' m')
            plt.axis(ax)

            filename=str(N)+' '+str(DEPTH_PENALTY)+'/cota'+str(COTA)+'.png'

            if SCATTER:
                filename='cortes horizontales con puntos 1 capa/'+filename
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                plt.savefig(filename, transparent=False, dpi=164, bbox_inches='tight')
            else:
                filename='cortes horizontales/'+filename
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                plt.savefig(filename, transparent=False, dpi=164, bbox_inches='tight')

            plt.clf()
            #plt.show()

COTA: 0
DEPTH_PENALTY: 1
N: 1
COTA: -5
DEPTH_PENALTY: 1
N: 1
COTA: -10
DEPTH_PENALTY: 1
N: 1
COTA: -15
DEPTH_PENALTY: 1
N: 1
COTA: -20
DEPTH_PENALTY: 1
N: 1
COTA: -25
DEPTH_PENALTY: 1
N: 1
COTA: -30
DEPTH_PENALTY: 1
N: 1
COTA: -40
DEPTH_PENALTY: 1
N: 1
COTA: -50
DEPTH_PENALTY: 1
N: 1
COTA: -60
DEPTH_PENALTY: 1
N: 1
COTA: -80
DEPTH_PENALTY: 1
N: 1
COTA: -100
DEPTH_PENALTY: 1
N: 1


<Figure size 432x288 with 0 Axes>